# Notebook per estrarre numpy arrays da un netcdf
Basta scegliere le variabili di interesse ed eseguire l'ultima cella, come ho lasciato da esempio

In [1]:
from netCDF4 import Dataset
import numpy as np
from pathlib import Path
import os
import pickle
# from wrf import getvar, latlon_coords, to_np

In [6]:
dirpath = r"C:\Users\giova\Desktop\hills"
variables = [
    'LFN',
    'FLINEINT',
    'F_ROS0',
    'F_ROSX',
    'F_ROSY',
    'F_ROS',
    'F_INT',
    'F_LINEINT',
    'F_LINEINT2',
    'HGT',
    'FWH',
    'FZ0',
    'FUEL_FRAC_BURNT', # = 1/w?
    'FGIP',            # total fuel load?
    'FGRNHFX',
    'T2',
    'U',
    'V',
    'UF',
    'VF'
] 
# ho segnato le variabili che potrebbero essere interessanti
# decidete voi cosa vale la pena tenere e cosa va aggiunto
# variables = ["ZSF"]

savefilepath = r"C:\Users\giova\Desktop\sim results\hill.pkl"


In [7]:
# root = Dataset(filepath, "r", format="NETCDF4")
# root.variables          # mostra tutte le variabili

In [8]:
def get_array_from_fire_subgrid(var_name, root):
    sr_x=int(len(root.dimensions['west_east_subgrid'])/(len(root.dimensions['west_east'])+1))
    sr_y=int(len(root.dimensions['south_north_subgrid'])/(len(root.dimensions['south_north'])+1))
    data=root.variables[var_name]
    return np.array(data[:,:-sr_y,:-sr_x].data), sr_x, sr_y


def process_data(root, var, expr=None):
    arr = np.array(root.variables[var][:].data)    
    lats = root.variables['XLAT'][0, :, 0]
    longs = root.variables['XLONG'][0, 0, :]    
    if 'south_north_subgrid' in root.variables[var].dimensions:
        arr, sr_x, sr_y = get_array_from_fire_subgrid(var, root)
        longs = np.linspace(longs.min(), longs.max(), len(root.dimensions['west_east_subgrid'])-sr_x), 
        lats = np.linspace(lats.min(), lats.max(), len(root.dimensions['south_north_subgrid'])-sr_y)

    return arr, longs, lats

In [13]:
with open(savefilepath, 'wb') as file:

    data = {}
    for var in variables:
        i = 0
        save = True
        for filename in os.listdir(dirpath):
            if filename[:6] == "wrfout": 
                print(f"Processing {filename}: {var}")
                root = Dataset(os.path.join(dirpath, filename), "r", format="NETCDF4")
                try:
                    arr_temp, lon, lat = process_data(root, var)
                except Exception as e:
                    save = False
                    print(f"Could not save variable {var}")
                if i == 0 : 
                    arr_tot = arr_temp
                    i+=1
                else: 
                    np.append(arr_tot, arr_temp)
               
        if save:
            a = {
                "data" : arr_tot,
                "description": root.variables[var].description,
                "coordinates": root.variables[var].coordinates,
                "shape": arr_tot.shape
            }
            data[var] = a

    pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)


Processing wrfout_d01_0001-01-01_00_00_00: LFN
Processing wrfout_d01_0001-01-01_00_00_00: FLINEINT
Processing wrfout_d01_0001-01-01_00_00_00: F_ROS0
Processing wrfout_d01_0001-01-01_00_00_00: F_ROSX
Processing wrfout_d01_0001-01-01_00_00_00: F_ROSY
Processing wrfout_d01_0001-01-01_00_00_00: F_ROS
Processing wrfout_d01_0001-01-01_00_00_00: F_INT
Processing wrfout_d01_0001-01-01_00_00_00: F_LINEINT
Processing wrfout_d01_0001-01-01_00_00_00: F_LINEINT2
Processing wrfout_d01_0001-01-01_00_00_00: HGT
Processing wrfout_d01_0001-01-01_00_00_00: FWH
Processing wrfout_d01_0001-01-01_00_00_00: FZ0
Processing wrfout_d01_0001-01-01_00_00_00: FUEL_FRAC_BURNT
Processing wrfout_d01_0001-01-01_00_00_00: FGIP
Processing wrfout_d01_0001-01-01_00_00_00: FGRNHFX
Processing wrfout_d01_0001-01-01_00_00_00: T2
Processing wrfout_d01_0001-01-01_00_00_00: U
Processing wrfout_d01_0001-01-01_00_00_00: V
Processing wrfout_d01_0001-01-01_00_00_00: UF
Processing wrfout_d01_0001-01-01_00_00_00: VF


In [14]:
data = []
with open(savefilepath, 'rb') as fp:
    data = pickle.load(fp)

data

{'LFN': {'data': array([[[4920.    , 4920.    , 4920.    , ..., 4920.    , 4920.    ,
           4920.    ],
          [4920.    , 4920.    , 4920.    , ..., 4920.    , 4920.    ,
           4920.    ],
          [4920.    , 4920.    , 4920.    , ..., 4920.    , 4920.    ,
           4920.    ],
          ...,
          [4920.    , 4920.    , 4920.    , ..., 4920.    , 4920.    ,
           4920.    ],
          [4920.    , 4920.    , 4920.    , ..., 4920.    , 4920.    ,
           4920.    ],
          [4920.    , 4920.    , 4920.    , ..., 4920.    , 4920.    ,
           4920.    ]],
  
         [[1118.6316, 1113.2618, 1107.8984, ..., 1065.4379, 1070.7458,
           1076.0612],
          [1115.9741, 1110.5914, 1105.2152, ..., 1062.6469, 1067.9688,
           1073.2979],
          [1113.3427, 1107.9473, 1102.5581, ..., 1059.8824, 1065.2183,
           1070.561 ],
          ...,
          [2179.5867, 2176.835 , 2174.0962, ..., 2155.1775, 2157.8066,
           2160.4495],
          [